# Data Query/Processing Task

Import VTK

In [48]:
from vtk import *   # import the VTK module

Load Data

In [49]:
reader = vtkXMLImageDataReader()            # create a reader
reader.SetFileName('Data/Isabel_2D.vti')    # set the file name
reader.Update()                             # update the reader
data = reader.GetOutput()                   # get the output of the reader

Query how many cells the dataset has

In [50]:
numCells = data.GetNumberOfCells()  # get the number of cells
print('Number of cells in the dataset: ',numCells)

Number of cells in the dataset:  62001


The dimensions of the dataset

In [51]:
dims = data.GetDimensions() # get the dimensions of the dataset
print('Dimensions of the dataset: ',dims)

Dimensions of the dataset:  (250, 250, 1)


The number of points present in the uniform grid of the data.

In [52]:
print('Total number of points in the grid: ', data.GetNumberOfPoints()) # get the number of points

Total number of points in the grid:  62500


Print the range of Pressure values present in the dataset.

In [53]:
dataArr = data.GetPointData().GetArray('Pressure')  # get the pressure array
print("Range of Pressure Values: ",dataArr.GetRange())  

Range of Pressure Values:  (-1434.8590087890625, 630.5694580078125)


Print the average Pressure value of the entire dataset, i.e., the Pressure array. For this, you will have
to access the Pressure values and compute the average of all points.

In [54]:
size = dataArr.GetDataSize()    # get the size of the array
avg=0

# calculate the average pressure value
for i in range(size):   
    avg += dataArr.GetTuple1(i)/size
print("Average pressure value: ", avg)

Average pressure value:  240.77722069091425


Extract a vtkCell object with cell id=0, i.e., the first cell. This is a quad cell, so the cell will have
four vertices and four edges, and one face.

In [55]:
cellId = 0      # cellId is the index of the cell

In [56]:
cell = data.GetCell(cellId) # get the cell with cellId
print('cell is a vtkCell object: ', bool(cell.IsA('vtkCell')))

cell is a vtkCell object:  True


Query the 4 corner points of the cell

In [57]:
# get the points of cell 
pid1 = cell.GetPointId(0)   
pid2 = cell.GetPointId(1)
pid3 = cell.GetPointId(3)
pid4 = cell.GetPointId(2)

Print the indices of the four corner vertices of the cell.

In [58]:
print('1D indices of the cell corner points:')
print(pid1,pid2,pid3,pid4) ## in counter-clockwise order

1D indices of the cell corner points:
0 1 251 250


Print the 3D coordinate of each vertex. Note that this is a 2D dataset, but all the points are stored
as 3D points in the VTK data file with a constant Z-coordinate value=25.

In [59]:
# get the 3d coordinates of the cell corners
print('3D Point locations of cell corners in counter clockwise order:')
print(data.GetPoint(pid1))
print(data.GetPoint(pid2))
print(data.GetPoint(pid3))
print(data.GetPoint(pid4))

3D Point locations of cell corners in counter clockwise order:
(0.0, 0.0, 25.0)
(1.0, 0.0, 25.0)
(1.0, 1.0, 25.0)
(0.0, 1.0, 25.0)


Compute the 3D coordinate of the cell center using its four corner vertices. The center location can
be computed as the average of the corner vertices. Again, note that the Z coordinate of the center
will be 25.

In [60]:
# Define a function to find the center
def findCenter(A, B, C, D):
    x = (A[0]+B[0]+C[0]+D[0])/4
    y = (A[1]+B[1]+C[1]+D[1])/4
    return (x,y, A[2])

In [61]:
center = findCenter(data.GetPoint(pid1), data.GetPoint(pid2), data.GetPoint(pid3), data.GetPoint(pid4)) 
print('Coordinates of the center of cell: ', center)

Coordinates of the center of cell:  (0.5, 0.5, 25.0)


Print the data/attribute value (Pressure) for all the four vertices of the extracted cell.

In [62]:
val_list = [dataArr.GetTuple1(pid1),dataArr.GetTuple1(pid2),dataArr.GetTuple1(pid3),dataArr.GetTuple1(pid4)]
print("List of Pressure Values: ",val_list)

List of Pressure Values:  [477.527587890625, 474.79827880859375, 467.60699462890625, 478.0115661621094]


Compute and print the mean (average) Pressure value at the cell center by averaging Pressure values
from the four cell vertices.

In [63]:
Pavg = sum(val_list)/len(val_list)
print("Average Pressure value at the center: ",Pavg)

Average Pressure value at the center:  474.4861068725586


# Visualization Task

Store one cell in the vtkPoints object

In [64]:
points = vtkPoints()    # create an empty vtkPoints object

# Add the points to the vtkPoints object
points.InsertNextPoint(data.GetPoint(pid1))
points.InsertNextPoint(data.GetPoint(pid2)) 
points.InsertNextPoint(data.GetPoint(pid3))
points.InsertNextPoint(data.GetPoint(pid4))

3

Create a polydata and add points to the polydata

In [65]:
poly_data = vtkPolyData()   # create an empty vtkPolyData object
poly_data.SetPoints(points) # add the points to the vtkPolyData object

Setup Colors

In [66]:
Colors = vtkUnsignedCharArray() # create an empty vtkUnsignedCharArray object
Colors.SetNumberOfComponents(3) # set the number of components to 3
Colors.SetName('Colors')        # set the name of the array to 'Colors'

# Add the colors to the vtkUnsignedCharArray object
Colors.InsertNextTuple3(255, 0, 0)     # red   
Colors.InsertNextTuple3(0, 255, 0)     # Green
Colors.InsertNextTuple3(66, 233, 245)  # Turquiose
Colors.InsertNextTuple3(0, 0, 255)     # Blue

# Add the colors to the vtkPolyData object
poly_data.GetPointData().SetScalars(Colors)     
poly_data.Modified()   

Apply the Vertex Glyph Filter on poly data

In [67]:
vertexGlyphFilter = vtkVertexGlyphFilter()  # create a vtkVertexGlyphFilter object
vertexGlyphFilter.AddInputData(poly_data)   # add the vtkPolyData object to the vtkVertexGlyphFilter object
vertexGlyphFilter.Update()                  # update the vtkVertexGlyphFilter object

Setup mapper and actor

In [68]:
mapper = vtkPolyDataMapper()                                    # create a vtkPolyDataMapper object
mapper.SetInputConnection(vertexGlyphFilter.GetOutputPort())    # set the input of the mapper to the output of the vtkVertexGlyphFilter object

actor = vtkActor()                                              # create a vtkActor object
actor.SetMapper(mapper)                                         # set the mapper of the actor to the vtkPolyDataMapper object
actor.GetProperty().SetPointSize(50)                            # set the size of the points

Setup render window, renderer, and interactor

In [69]:
renderer = vtkRenderer()                               # create a vtkRenderer object
renderer.SetBackground(1,1,1)                          # set the background color of the renderer to white
renderWindow = vtkRenderWindow()                       # create a vtkRenderWindow object
renderWindow.SetSize(800,800)                          # set the size of the render window
renderWindow.AddRenderer(renderer)                     # add the renderer to the render window
renderWindowInteractor = vtkRenderWindowInteractor()   # create a vtkRenderWindowInteractor object
renderWindowInteractor.SetRenderWindow(renderWindow)   # set the render window of the render window interactor to the vtkRenderWindow object
renderer.AddActor(actor)                               # add the actor to the renderer

Finally render the object

In [ ]:
renderWindow.Render()           # render the scene
renderWindowInteractor.Start()  # start the interaction